In [1]:
import json

class ToDictMixin:
    def to_dict(self):
        return self._traverse_dict(self.__dict__)

    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output

    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value
            
class JsonMixin:
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)

    def to_json(self):
        return json.dumps(self.to_dict())

class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [
            Machine(**kwargs) for kwargs in machines]

class Switch(ToDictMixin, JsonMixin):
    def __init__(self, ports=None, speed=None):
        self.ports = ports
        self.speed = speed

class Machine(ToDictMixin, JsonMixin):
    def __init__(self, cores=None, ram=None, disk=None):
        self.cores = cores
        self.ram = ram
        self.disk = disk

serialized = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12},
        {"cores": 2, "ram": 4e9, "disk": 500e9}
    ]
}"""


In [ ]:

deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()
assert json.loads(serialized) == json.loads(roundtrip)

In [2]:
deserialized

In [4]:
roundtrip

'{"switch": {"ports": 5, "speed": 1000000000.0}, "machines": [{"cores": 8, "ram": 32000000000.0, "disk": 5000000000000.0}, {"cores": 4, "ram": 16000000000.0, "disk": 1000000000000.0}, {"cores": 2, "ram": 4000000000.0, "disk": 500000000000.0}]}'

In [5]:
json.loads(roundtrip)

{'switch': {'ports': 5, 'speed': 1000000000.0},
 'machines': [{'cores': 8, 'ram': 32000000000.0, 'disk': 5000000000000.0},
  {'cores': 4, 'ram': 16000000000.0, 'disk': 1000000000000.0},
  {'cores': 2, 'ram': 4000000000.0, 'disk': 500000000000.0}]}

In [6]:
json.loads(serialized)

{'switch': {'ports': 5, 'speed': 1000000000.0},
 'machines': [{'cores': 8, 'ram': 32000000000.0, 'disk': 5000000000000.0},
  {'cores': 4, 'ram': 16000000000.0, 'disk': 1000000000000.0},
  {'cores': 2, 'ram': 4000000000.0, 'disk': 500000000000.0}]}